## 1. Loading prereqs

In [ ]:
try:
    from google.colab import drive
    IN_COLAB = True
    print("Running on Google Colab. ")
except:
    IN_COLAB = False
    print("Not running on Google Colab. ")

In [ ]:
if IN_COLAB:
    !pip install facenet-pytorch  # fornisce modelli pre-addestrati PyTorch per compiti di riconoscimento facciale
    !pip install Pillow # aggiunge il supporto per l'apertura, la manipolazione e il salvataggio di molti diversi formati di file immagine.

## 2. Load NN1

#### Load pre-trained model





In [ ]:
# utilizzo la libreria facenet_pytorch per caricare il modello InceptionResnetV1 preaddestrato sul dataset VGGFace2 e abilitare la classificazione.
from facenet_pytorch import InceptionResnetV1, MTCNN
import torch
# image_size = 160
# margin = 0
# mtcnn = MTCNN(image_size,margin)

resnet = InceptionResnetV1(pretrained='vggface2').eval()
resnet.classify = True

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))
resnet = resnet.to(device)



#### Loading labels of model

In [ ]:
import numpy as np
import tensorflow as tf

# Il modello è addestrato sulle seguenti Labels:
# Carico le labels del dataset VGGFACE
fpath = tf.keras.utils.get_file('rcmalli_vggface_labels_v2.npy',
                             "https://github.com/rcmalli/keras-vggface/releases/download/v2.0/rcmalli_vggface_labels_v2.npy",
                             cache_subdir="./")
LABELS = np.load(fpath) # List of name
# Clean list of name
#name_LABELS={}
for i in range(len(LABELS)):
  LABELS[i] = LABELS[i].strip().replace(' ', '').replace('"', '')
  #name_LABELS[LABELS[i]]=i

## 3. Load Test Set

#### Connect to Drive

In [ ]:
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive/')

#### Load the label of Test Set (for NN1)

In [ ]:
# set the path for the dataset

if IN_COLAB:
    path_dataset = "/content/drive/Shareddrives/AI4CYBSEC/face_dataset"
else:
    path_dataset = "./face_dataset"
identity_meta_NN1_name = "meta_identity_NN1.csv"

import pandas as pd
import os

path_identity_csv =os.path.join(path_dataset,identity_meta_NN1_name)
identity_meta_NN1 = pd.read_csv(path_identity_csv)


## 4. Mapping different label encoding

In [ ]:
# I want a dictonary related to the label of the Test Set that map the name of celebrities with label associated
name_to_id = {}
id_to_name = {}
for index, row in identity_meta_NN1.iterrows():
    # Ora puoi accedere ai valori di ogni riga come segue:
    class_id = row['Class_ID']
    name = row['Name']
    name_to_id[name]=class_id
    id_to_name[class_id] = name



## 5. Dataset Class

In [ ]:
from PIL import Image
import os
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

class VGGFace2Dataset(Dataset):
    def __init__(self, root_dir, image_size=(160, 160), transform=None):
        self.root_dir = root_dir
        self.image_size = image_size
        self.transform = transform

        # List of files in the dataset
        self.file_list = []
        for root, dirs, files in os.walk(self.root_dir):
            for file in files:
                self.file_list.append(os.path.join(root, file))

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        img_path = self.file_list[idx]
        img = Image.open(img_path).resize(self.image_size)

        # Extract the label from the file path
        label = os.path.split(os.path.dirname(img_path))[-1]

        if self.transform:
            img = self.transform(img)

        return img, label




## 6. Evaluate model NN1


### Utility Function for NN1 with mapping labels

In [ ]:

from PIL import Image
from torchvision import transforms
import torch
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm

def load_image(file_path):
    """ carica un'immagine da un percorso e la apre come un'immagine utilizzando Image.open dal modulo Pillow.
    Successivamente, ridimensiona l'immagine a dimensioni 160x160 pixel e la converte in un tensore utilizzando
    transforms.ToTensor() dal modulo torchvision.transforms.
    Infine, restituisce sia il tensore dell'immagine che l'immagine aperta.
    """
    rsz = Image.open(file_path).resize((160, 160))
    tns = transforms.ToTensor()(rsz)
    return tns, rsz
def make_inference(model, image_tensors, name_to_id, device):
    """
    Takes input image tensor and returns the label associated with the network's prediction.
    """
    # Move image tensors to the specified device
    image_tensors = image_tensors.to(device)

    probs = model(image_tensors)
    #print("probs", probs)

    # Get the number of elements along the first dimension
    num_elements = probs.size(0)

    # Initialize two lists to store the argmax
    argmax_list_1 = []
    argmax_list_2 = []

    # Compute argmax for each element along the first dimension
    for i in range(num_elements):
        target_class = np.array(probs[i].detach().cpu().numpy()).argmax()  # Move to CPU for numpy operations
        argmax_list_1.append(name_to_id[LABELS[target_class]])
        argmax_list_2.append(target_class)

    return argmax_list_1, argmax_list_2

def validate(dataset, model, name_to_id, device):
    """
    Validates a model on a dataset and returns the accuracy.

    Args:
        dataset: Dataloader to validate the model on.
        model: Model to validate.
        device: Device to perform inference on.

    Returns:
        accuracy: Accuracy of the model on the dataset.
    """
    model.eval()
    correct_predictions = 0
    total_samples = len(dataset) * dataset.batch_size

    with torch.no_grad():  # Disable gradient calculation
        for images, labels in tqdm(dataset, desc="Validating model"):
            #images = mtcnn(images)
            predicted_classes, _ = make_inference(model, images, name_to_id, device)
            correct_predictions += sum(pred == label for pred, label in zip(predicted_classes, labels))

    # Compute accuracy
    accuracy = correct_predictions / total_samples
    return accuracy


def plot_image(original_image, original_label):
  """
  prende in ingresso le PIL.Image del campione originale e del corrispondete adversarial sample e li plotta
  """
  plt.figure()
  plt.matshow(original_image)
  plt.title("Model Prediction: {}".format(original_label))
  plt.show()

### Validation on Clean Data

In [ ]:
# Create transform for image resizing and normalization
data_transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor()
])

if IN_COLAB:
    test_set_path = "/content/drive/Shareddrives/AI4CYBSEC/face_dataset/test_set_MTCNN"
else:
    test_set_path = "./face_dataset/test_set_MTCNN"
# Define dataset
dataset = VGGFace2Dataset(root_dir=test_set_path, transform=data_transform)

# Check the length of the dataset
print("Dataset length:", len(dataset))

# Create DataLoader
batch_size = 1
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

acc = validate(dataloader, resnet, name_to_id, device)
print("\n"+str(acc))
